In [ ]:
# --------- Importation des packages ----------
import os, io, gzip, boto3, pandas as pd, numpy as np
from dotenv import load_dotenv

# --------- Chargement des variables d'environnement ----------

load_dotenv()
AWS_REGION = os.getenv("AWS_REGION")
S3_BUCKET  = os.getenv("S3_BUCKET_STAGING")

# --------- Connexion au bucket S3 ----------

s3 = boto3.client("s3", region_name=AWS_REGION)

# --------- Définitions des fonctions ----------

# Fonction pour récupérer les clés dans le bucket S3
def iter_s3_keys(bucket, suffix=None, prefix=None):
    token = None
    while True:
        kw = {"Bucket": bucket}
        if token: kw["ContinuationToken"] = token
        if prefix: kw["Prefix"] = prefix
        resp = s3.list_objects_v2(**kw)
        for c in resp.get("Contents", []):
            k = c["Key"]
            if not suffix or k.endswith(suffix):
                yield k
        if not resp.get("IsTruncated"):
            break
        token = resp.get("NextContinuationToken")

# Fonction pour charger les fichiers dans un dataframe
def read_jsonl_gz_from_s3(bucket, key):
    obj = s3.get_object(Bucket=bucket, Key=key)
    with gzip.GzipFile(fileobj=obj["Body"]) as gz:
        # pd.read_json accepte un file-like texte
        df = pd.read_json(io.TextIOWrapper(gz, encoding="utf-8"), lines=True)
    return df

# Fonction de nettoyage des données
def clean_numeric(series, unit=None, factor=1.0, decimals=None):
    s = series.astype(str).str.replace("\xa0"," ", regex=False).str.strip()
    if unit: s = s.str.replace(unit, "", regex=False).str.strip()
    s = pd.to_numeric(s, errors="coerce") * factor
    if decimals is not None: s = s.round(decimals)
    return s

# --------- Chargement du fichier infoclimat depuis S3 ----------
key_info = max([k for k in iter_s3_keys(S3_BUCKET, suffix=".jsonl.gz", prefix="greenandcoop-staging/infoclimat/")])
df_raw_infoclimat = read_jsonl_gz_from_s3(S3_BUCKET, key_info)

raw = df_raw_infoclimat["_airbyte_data"].iloc[0]
hourly = raw["hourly"]
stations = pd.DataFrame(raw["stations"])

# Aplatir hourly (une ligne = un relevé)
dfs = []
for sid, recs in hourly.items():
    if sid == "_params": 
        continue
    df = pd.DataFrame.from_records(recs)
    df["id_station"] = sid
    dfs.append(df)
df_infoclimat = pd.concat(dfs, ignore_index=True)

# --------- Transformation du Dataframe infoclimat ----------

# Joindre métadonnées stations
stations = stations.rename(columns={"id":"id_station", "name":"station_name"})
df_infoclimat = df_infoclimat.merge(stations, on="id_station", how="left")

# Types et découpes temporelles
df_infoclimat = df_infoclimat.rename(columns={"id_station":"station_id"})
for c in ["latitude","longitude","elevation"]:
    df_infoclimat[c] = pd.to_numeric(df_infoclimat[c], errors="coerce")

df_infoclimat["dh_utc"] = pd.to_datetime(df_infoclimat["dh_utc"], utc=True, errors="coerce")
df_infoclimat["date"] = df_infoclimat["dh_utc"].dt.tz_convert("Europe/Paris").dt.normalize()
df_infoclimat["time"] = df_infoclimat["dh_utc"].dt.tz_convert("Europe/Paris").dt.strftime("%H:%M:%S")
df_infoclimat = df_infoclimat.drop(columns=["dh_utc"])

num_cols_info = ["temperature","point_de_rosee","visibilite","humidite",
                 "vent_direction","vent_moyen","vent_rafales","pression",
                 "pluie_1h","pluie_3h","neige_au_sol","nebulosite"]
for c in num_cols_info:
    if c in df_infoclimat:
        df_infoclimat[c] = pd.to_numeric(df_infoclimat[c], errors="coerce")

# --------- Chargement des fichiers CSV depuis S3 ----------
key_csv = max([k for k in iter_s3_keys(S3_BUCKET, suffix=".jsonl.gz", prefix="greenandcoop-staging/greenandcoop-csvfiles/")])
df_csv_raw = read_jsonl_gz_from_s3(S3_BUCKET, key_csv)
df_csv = pd.DataFrame(df_csv_raw["_airbyte_data"].to_dict()).T.reset_index(drop=True)
df_csv.columns = df_csv.columns.str.lower()

# --------- Transformation du Dataframe CSV ----------
rename_map = {
    "dew point":"point_de_rosee","humidity":"humidite","wind":"vent_direction",
    "speed":"vent_moyen","gust":"vent_rafales","pressure":"pression",
    "precip. rate.":"precipitation_moyenne","precip. accum.":"precipitation_acc",
    "solar":"flux_solaire"
}
df_csv = df_csv.rename(columns=rename_map)

# Conversions CSV
for c in ["latitude","longitude","elevation","uv"]:
    if c in df_csv: df_csv[c] = pd.to_numeric(df_csv[c], errors="coerce")
df_csv["date"] = pd.to_datetime(df_csv["date"], format="%d%m%y", errors="coerce")
df_csv["time"] = pd.to_datetime(df_csv["time"], format="%H:%M:%S", errors="coerce").dt.strftime("%H:%M:%S")

# °F -> °C
df_csv["temperature"]     = (clean_numeric(df_csv["temperature"], unit="°F")     - 32)*5/9
df_csv["point_de_rosee"]  = (clean_numeric(df_csv["point_de_rosee"], unit="°F") - 32)*5/9
df_csv[["temperature","point_de_rosee"]] = df_csv[["temperature","point_de_rosee"]].round(1)

# % / vent / pression / pluie / flux
df_csv["humidite"]              = clean_numeric(df_csv["humidite"], unit="%", decimals=0)
wind_map = {"N":0,"North":0,"NNE":22.5,"NE":45,"ENE":67.5,"E":90,"East":90,"ESE":112.5,
            "SE":135,"SSE":157.5,"S":180,"South":180,"SSW":202.5,"SW":225,"WSW":247.5,
            "W":270,"West":270,"WNW":292.5,"NW":315,"NNW":337.5}
df_csv["vent_direction"] = df_csv["vent_direction"].map(wind_map)
df_csv["vent_moyen"]     = clean_numeric(df_csv["vent_moyen"],  unit="mph", factor=1.60934, decimals=1)
df_csv["vent_rafales"]   = clean_numeric(df_csv["vent_rafales"],unit="mph", factor=1.60934, decimals=1)
df_csv["pression"]       = clean_numeric(df_csv["pression"],     unit="in",  factor=33.8639, decimals=1)
df_csv["precipitation_moyenne"] = clean_numeric(df_csv["precipitation_moyenne"], unit="in", factor=25.4, decimals=1)
df_csv["precipitation_acc"]     = clean_numeric(df_csv["precipitation_acc"],     unit="in", factor=25.4, decimals=1)
df_csv["flux_solaire"]   = clean_numeric(df_csv["flux_solaire"], unit="w/m²")

# --------- Concaténation des Dataframes ----------
schema = [
    "station_id","station_name","city","latitude","longitude","elevation","software",
    "date","time","type","license","temperature","point_de_rosee","visibilite","humidite",
    "vent_direction","vent_moyen","vent_rafales","pression","pluie_1h","pluie_3h",
    "precipitation_moyenne","precipitation_acc","uv","flux_solaire","neige_au_sol",
    "nebulosite","temps_omm"
]

# Harmoniser les deux DF au schéma cible
df_infoclimat = df_infoclimat.reindex(columns=schema)
df_csv        = df_csv.reindex(columns=schema)

df_final = pd.concat([df_infoclimat, df_csv], ignore_index=True)
df_final = df_final.replace({None:np.nan})

TypeError: Cannot convert tz-naive timestamps, use tz_localize to localize

In [11]:
df_final

,station_id,station_name,city,latitude,longitude,elevation,software,date,time,type,...,pression,pluie_1h,pluie_3h,precipitation_moyenne,precipitation_acc,uv,flux_solaire,neige_au_sol,nebulosite,temps_omm
0,07015,Lille-Lesquin,NaN,50.575,3.092,47,NaN,2024-10-05 00:00:00+02:00,02:00:00,synop,...,1020.7,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,07015,Lille-Lesquin,NaN,50.575,3.092,47,NaN,2024-10-05 00:00:00+02:00,03:00:00,synop,...,1020.6,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,07015,Lille-Lesquin,NaN,50.575,3.092,47,NaN,2024-10-05 00:00:00+02:00,04:00:00,synop,...,1020.3,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10
3,07015,Lille-Lesquin,NaN,50.575,3.092,47,NaN,2024-10-05 00:00:00+02:00,05:00:00,synop,...,1020.1,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,10
4,07015,Lille-Lesquin,NaN,50.575,3.092,47,NaN,2024-10-05 00:00:00+02:00,06:00:00,synop,...,1019.9,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4959,IICHTE19,WeerstationBS,Ichtegem,51.092,2.999,15,EasyWeather_V1.6.6,2024-10-06 00:00:00,23:39:00,NaN,...,991.5,NaN,NaN,0.0,2.3,0.0,0.0,NaN,NaN,NaN
4960,IICHTE19,WeerstationBS,Ichtegem,51.092,2.999,15,EasyWeather_V1.6.6,2024-10-06 00:00:00,23:44:00,NaN,...,991.2,NaN,NaN,0.0,2.3,0.0,0.0,NaN,NaN,NaN
4961,IICHTE19,WeerstationBS,Ichtegem,51.092,2.999,15,EasyWeather_V1.6.6,2024-10-06 00:00:00,23:49:00,NaN,...,991.2,NaN,NaN,0.0,2.3,0.0,0.0,NaN,NaN,NaN
4962,IICHTE19,WeerstationBS,Ichtegem,51.092,2.999,15,EasyWeather_V1.6.6,2024-10-06 00:00:00,23:54:00,NaN,...,991.5,NaN,NaN,0.0,2.3,0.0,0.0,NaN,NaN,NaN


In [7]:
df_final.dtypes

station_id                object
station_name              object
city                      object
latitude                 float64
longitude                float64
elevation                  int64
software                  object
date                      object
time                      object
type                      object
license                   object
temperature              float64
point_de_rosee           float64
visibilite               float64
humidite                 float64
vent_direction           float64
vent_moyen               float64
vent_rafales             float64
pression                 float64
pluie_1h                 float64
pluie_3h                 float64
precipitation_moyenne    float64
precipitation_acc        float64
uv                       float64
flux_solaire             float64
neige_au_sol             float64
nebulosite               float64
temps_omm                 object
dtype: object

In [ ]:
# --------- Préparation des données pour MongoDB ----------

# 2. Conversion pandas NaN / NaT en None pour compatibilité Mongo
def to_mongo_records(df: pd.DataFrame):
    for rec in df.to_dict("records"):
        for k, v in rec.items():
            if pd.isna(v):  # attrape NaN, NaT, pd.NA
                rec[k] = None
        yield rec

mongo_ready = list(to_mongo_records(df_final))

{'station_id': '07015', 'station_name': 'Lille-Lesquin', 'city': None, 'latitude': 50.575, 'longitude': 3.092, 'elevation': 47, 'software': None, 'date': Timestamp('2024-10-05 00:00:00+0200', tz='Europe/Paris'), 'time': '02:00:00', 'type': 'synop', 'license': {'license': 'Etalab Open License', 'url': 'https://www.etalab.gouv.fr/licence-ouverte-open-licence', 'source': 'Meteo-France via infoclimat.fr', 'metadonnees': 'https://donneespubliques.meteofrance.fr/metadonnees_publiques/fiches/fiche_59343001.pdf'}, 'temperature': 7.6, 'point_de_rosee': 5.9, 'visibilite': 6000.0, 'humidite': 89.0, 'vent_direction': 90.0, 'vent_moyen': 3.6, 'vent_rafales': 7.2, 'pression': 1020.7, 'pluie_1h': 0.0, 'pluie_3h': 0.0, 'precipitation_moyenne': None, 'precipitation_acc': None, 'uv': None, 'flux_solaire': None, 'neige_au_sol': None, 'nebulosite': None, 'temps_omm': None, 'date_str': '2024-10-05', 'time_str': '02:00:00', 'ts_utc': Timestamp('2024-10-05 00:00:00')}
